# 004 Pre analysis before clustering
* Filter pings on highways and major roads
* Indentify home locations for users at a grid level

In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Check package versions
import types 
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

import pkg_resources
root_packages = [i.split('.', 1)[0] for i in list(imports())] 
for m in pkg_resources.working_set:
    if m.project_name.lower() in root_packages:
        print (m.project_name, m.version)

('pandas', '0.24.0')
('numpy', '1.15.4')
('matplotlib', '2.2.3')


In [ ]:
def thsp(number):
    print format(number,",")

## Load data after labeling pings on highways

In [ ]:
fp = '../../../share/output/Houston/houston_aug_sep_2017_highway.csv'
df = pd.read_csv(fp)

In [ ]:
thsp(len(df))
thsp(len(df[df['highway']==False]))

In [ ]:
print thsp(df['ad_id'].nunique())
print thsp(df['activity'].sum())

### Drop pings on highways

In [ ]:
# Dropping pings on highway
df = df[df['highway']==False]

In [ ]:
df = df.reset_index(drop=True)
df.head()

In [ ]:
thsp(df['ad_id'].nunique())

# Data Preparation
### Convert timestamp to Centeral Time
* Push 5 hours to left side (-5 hours)

In [ ]:
df['date'] = df['timestamp'].str[0:10] # date part
df['hour'] = df['timestamp'].str[11:13] # hour part
df['minute'] = df['timestamp'].str[14:16] # minute part

df['dt'] = df['date'] + ' ' + df['hour'] + ':' + df['minute'] + ':00'
df['dt'] = pd.to_datetime(df['dt'], errors='coerce')
df['dt_ct'] = df['dt'] - pd.Timedelta(hours=5)

df.head()

### Round lat and lon to 2 decimal places

In [ ]:
df['lon_round'] = df['lon'].round(2)
df['lat_round'] = df['lat'].round(2)
df.head(2)

### Create 'cell_id'
* The grid cell id = 'lon_round' + 'lat_round' as string format

In [ ]:
df['cell_id'] = df['lon_round'].astype(str) + ',' + df['lat_round'].astype(str)
df[['ad_id', 'dt_ct', 'dt', 'date', 'hour', 'minute', 'lon_round', 'lat_round', 'cell_id']].head()

# Analysis
### Entire data: Identifying daily nighttime locations (home locations)

In [ ]:
df[['ad_id', 'dt_ct', 'lon_round', 'lat_round', 'cell_id']].head()

In [ ]:
nighttime = ['00', '01', '02', '03', '04', '05', '06', '20', '21', '22', '23']

date_list_pre = ['2017-08-01', '2017-08-02', '2017-08-03', '2017-08-04', '2017-08-05', '2017-08-06',
                '2017-08-07', '2017-08-08', '2017-08-09', '2017-08-10', '2017-08-11', '2017-08-12',
                '2017-08-13', '2017-08-14', '2017-08-15', '2017-08-16']

date_list_post = ['2017-08-17', '2017-08-18', '2017-08-19', '2017-08-20', '2017-08-21', 
                 '2017-08-22', '2017-08-23', '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
                 '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31', '2017-09-01', '2017-09-02',
                 '2017-09-03', '2017-09-04', '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08', 
                 '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12', '2017-09-13', '2017-09-14',
                 '2017-09-15', '2017-09-16', '2017-09-17', '2017-09-18', '2017-09-19', '2017-09-20',
                 '2017-09-21', '2017-09-22', '2017-09-23', '2017-09-24', '2017-09-25', '2017-09-26',
                 '2017-09-27', '2017-09-28', '2017-09-29', '2017-09-30']

# Extract data points during nighttime
df_night = df[df['dt_ct'].astype(str).str[11:13].isin(nighttime)]

# Number of data points during nighttime and daytime
thsp(len(df_night))
thsp(len(df)-len(df_night))

# Number of total pings during nighttime and daytime
thsp(df_night['activity'].sum())
thsp(df['activity'].sum() - df_night['activity'].sum())

# For nighttime pings after midnight (0am), date need to be shifted
# because for example, the nighttime for 2017-08-01 is from 19:00pm 2017-08-01 to 7:59am 2017-08-02

df_night['date_revised'] = df_night['dt_ct'].dt.date
df_night['date_revised'][df_night['dt_ct'].astype(str).str[11:13].isin(['00', '01', '02', '03', '04', '05', '06'])] = df_night['date_revised'] - pd.Timedelta(days=1)
df_night = df_night[df_night['date_revised']!='2017-07-31']

df_night['date_revised_str'] = df_night['date_revised'].astype(str)

df_pre = df_night[df_night['date_revised_str'].isin(date_list_pre)].reset_index(drop=True)
df_post = df_night[df_night['date_revised_str'].isin(date_list_post)].reset_index(drop=True)

In [ ]:
df_pre = df_pre[['ad_id', 'dt_ct', 'date_revised', 'date_revised_str', 'lon_round', 'lat_round', 'cell_id', 'activity']]
df_post = df_post[['ad_id', 'dt_ct', 'date_revised', 'date_revised_str', 'lon_round', 'lat_round', 'cell_id', 'activity']]

In [ ]:
df_pre.head(2)

In [ ]:
df_post.head(2)

# Compute mode of 
* pre-hurricane
* daily locations for post-hurricane
### 1) Mode of pre-hurricane

In [ ]:
df_pre['count'] = 1

### One single mode of pre-hurricane

In [ ]:
df_pre_group = df_pre[['ad_id', 'cell_id', 'count', 'activity']].groupby(['ad_id', 'cell_id']).sum().reset_index()
df_pre_group.head(2)

In [ ]:
thsp(len(df_pre_group['ad_id'].unique()))
thsp(len(df_pre_group[df_pre_group['count']==1]['ad_id'].unique()))

In [ ]:
user_list_pre = df_pre_group['ad_id'].unique().tolist()
thsp(len(user_list_pre))

In [ ]:
df_pre_group2 = df_pre_group.sort_values(by='count', ascending=False).drop_duplicates(['ad_id'])
thsp(len(df_pre_group2))

thsp(len(df_pre_group2[df_pre_group2['count']==1]))
thsp(len(df_pre_group2[df_pre_group2['count']>1]))
thsp(len(df_pre_group2[(df_pre_group2['count']==1)&(df_pre_group2['activity']==1)]))

In [ ]:
df_pre_group2.head(2)

In [ ]:
# Replace 'cell_id' to NaN if 'count == 1' (only one ping is not enough to determine home locations)
df_pre_group2['cell_id'][(df_pre_group2['count']==1)&(df_pre_group2['activity']==1)] = np.nan
df_pre_group2 = df_pre_group2.reset_index(drop=True)
thsp(df_pre_group2['cell_id'].isnull().sum())

In [ ]:
# df_pre_group2.to_csv('../../outputs/nighttime_mode_cell_pre_baseline.csv', index=False)

### Daily mode 'cell_id's of  pre-hurricane

In [ ]:
df_pre_group_d = df_pre[['ad_id', 'cell_id', 'count', 'date_revised', 'activity']].groupby(['ad_id', 'cell_id', 'date_revised']).sum().reset_index()
df_pre_group_d.head()

In [ ]:
print (df_pre_group_d['date_revised'].max())

### Sorting --> then dropping dupliates might lose actual home 'cell_id'
* For example, at 08/23, user AAA has 1 ping at X cell and 1 ping at Y cell
* Actual home cell of the user AAA is Y, based on the baseline result
* Then, if dataframe sorting and dropping dupliates method drop Y, it would be problematic
* Therefore, if there are the same counts at different cells, the baseline location (if it's in the candidates) should be prioritized

### Method
* Join the baseline as a new column
* If 'cell_id' == 'baseline', plus 1 for "count" column (Weighting a cell which is the same to the baseline)
* Then sort and drop_dupliates

In [ ]:
# Load baseline df
baseline = pd.read_csv('../../outputs/nighttime_mode_cell_pre_baseline.csv')
baseline.head(2)

In [ ]:
baseline.columns = ['ad_id', 'baseline', 'count_baseline', 'activity_baseline']

# Join df_pre_group with baseline
thsp(len(df_pre_group_d))
df_pre_group_base = pd.merge(df_pre_group_d, baseline, on='ad_id', how='left')
thsp(len(df_pre_group_base))

In [ ]:
df_pre_group_base['count'][df_pre_group_base['cell_id']==df_pre_group_base['baseline']] = df_pre_group_base['count']+1
thsp(len(df_pre_group_base))
thsp(len(df_pre_group_base[df_pre_group_base['count']==1]))

df_pre_group_base['cell_id'][df_pre_group_base['count']==1] = np.nan
df_pre_group_base = df_pre_group_base.reset_index(drop=True)
thsp(df_pre_group_base['cell_id'].isnull().sum())

In [ ]:
df_pre_group_base.tail()

In [ ]:
# Then sort (descending by 'count') and drop_dupliates by ['ad_id', 'date']
df_pre_group_d_2 = df_pre_group_base.sort_values(by='count', ascending=False).drop_duplicates(['ad_id', 'date_revised']).reset_index(drop=True)
thsp(len(df_pre_group_d_2))
df_pre_group_d_2.head()

In [ ]:
thsp(df_pre_group_d_2['ad_id'].nunique())

In [ ]:
# df_pre_group_d_2.to_csv('../../outputs/nighttime_mode_cell_pre_daily.csv', index=False)

### Daily mode cells of post-hurricane

In [ ]:
df_post.head()

In [ ]:
df_post['count'] = 1

df_post_group = df_post[['ad_id', 'cell_id', 'date_revised', 'count', 'activity']].groupby(['ad_id', 'cell_id', 'date_revised']).sum().reset_index()
df_post_group.head(2)

In [ ]:
user_list_post = df_post_group['ad_id'].unique().tolist()
thsp(len(user_list_post))

In [ ]:
# Join df_post_group with baseline
thsp(len(df_post_group))
df_post_group_base = pd.merge(df_post_group, baseline, on='ad_id', how='left')
thsp(len(df_post_group_base))
df_post_group_base.head(2)

In [ ]:
df_post_group_base['count'][df_post_group_base['cell_id']==df_post_group_base['baseline']] = df_post_group_base['count']+1

thsp(len(df_post_group_base))
thsp(len(df_post_group_base[df_post_group_base['count']==1]))

df_post_group_base['cell_id'][df_post_group_base['count']==1] = np.nan
df_post_group_base = df_post_group_base.reset_index(drop=True)
thsp(df_post_group_base['cell_id'].isnull().sum())

df_post_group_base.head()

In [ ]:
# Then sort (decending by 'count') and drop_dupliates by ['ad_id', 'date_revised']
df_post_group_2 = df_post_group_base.sort_values('count', ascending=False).drop_duplicates(['ad_id', 'date_revised']).reset_index(drop=True)
thsp(len(df_post_group_2))
df_post_group_2.head(2)

In [ ]:
thsp(len(df_post_group_2['ad_id'].unique()))

In [ ]:
# df_post_group_2.to_csv('../../outputs/nighttime_mode_cell_post_daily.csv', index=False)